In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import torch
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:




# Define paths
input_dirs = [
    '/content/drive/My Drive/GAN/Real',
    '/content/drive/My Drive/Dall_E',
    '/content/drive/My Drive/stable_difuser_images'
]
label_folder = '/content/drive/My Drive/processed_images/centerbased/labels'
center_based_model_folder = '/content/drive/My Drive/processed_images/centerbased'

In [ ]:
# Load YOLOv5 large model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:293: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 130.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 273.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 273.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 8.5s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-25 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|██████████| 89.3M/89.3M [00:00<00:00, 444MB/s]

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 


# center based

In [ ]:
# Define paths for each input directory
input_dirs = {
    'GAN/Real': '/content/drive/My Drive/GAN/Real',
    'Dall_E': '/content/drive/My Drive/Dall_E',
    'Stable_Diffuser': '/content/drive/My Drive/stable_difuser_images'
}
label_folder = '/content/drive/My Drive/processed_images/centerbased/labels'
center_based_model_folder = '/content/drive/My Drive/processed_images/centerbased'

# Create directories if they don't exist
images_with_human_on_chair_folder = os.path.join(center_based_model_folder, 'images_with_human_on_chair')
annotations_with_human_on_chair_folder = os.path.join(center_based_model_folder, 'annotations_with_human_on_chair')

os.makedirs(images_with_human_on_chair_folder, exist_ok=True)
os.makedirs(annotations_with_human_on_chair_folder, exist_ok=True)

# Define relevant classes for objects that a human can sit on next to a pool
relevant_classes = [0, 56, 57, 58]  # Include humans and objects they can sit on

# Function to convert YOLO bbox to normal bbox
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x = int((x_center - width / 2) * img_width)
    y = int((y_center - height / 2) * img_height)
    w = int(width * img_width)
    h = int(height * img_height)
    return [x, y, w, h]

# Function to convert normal bbox to YOLO bbox
def bbox_to_yolo(bbox, img_width, img_height):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    width = w / img_width
    height = h / img_height
    return [x_center, y_center, width, height]

# Function to check if a human is on a chair
def is_human_on_chair(human_bbox, chair_bbox):
    hx, hy, hw, hh = human_bbox
    cx, cy, cw, ch = chair_bbox
    # Check if the center of the human is within the chair bounding box
    return (cx <= hx + hw / 2 <= cx + cw) and (cy <= hy + hh / 2 <= cy + ch)

# Function to detect objects using YOLOv5 model
def detect_objects(image_path):
    results = model(image_path)
    return results.xyxy[0].cpu().numpy()

# Function to plot bounding boxes on an example image and save it
def plot_and_save_image_with_bboxes(image_path):
    image_name = os.path.basename(image_path)
    label_path = os.path.join(label_folder, image_name.replace('.jpg', '.txt'))

    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return 0, 0, 0

    img_height, img_width = img.shape[:2]

    detections = detect_objects(image_path)
    humans = []
    chairs = []

    for det in detections:
        class_id, x1, y1, x2, y2, confidence = int(det[5]), int(det[0]), int(det[1]), int(det[2]), int(det[3]), float(det[4])
        bbox = [x1, y1, x2 - x1, y2 - y1]
        if class_id == 0:
            humans.append(bbox)
        elif class_id in relevant_classes:
            chairs.append(bbox)

    human_on_chair_count = 0
    new_annotations = []

    for human_bbox in humans:
        for chair_bbox in chairs:
            if is_human_on_chair(human_bbox, chair_bbox):
                human_on_chair_count += 1
                cv2.rectangle(img, (human_bbox[0], human_bbox[1]), (human_bbox[0] + human_bbox[2], human_bbox[1] + human_bbox[3]), (255, 0, 0), 3)
                cv2.putText(img, 'Human on Chair', (human_bbox[0], human_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                # Save the new annotation for human on chair
                new_annotations.append((2, bbox_to_yolo(human_bbox, img_width, img_height)))
                break

    if human_on_chair_count > 0:
        # Save the image with bounding boxes and annotations
        save_path = os.path.join(images_with_human_on_chair_folder, 'human_on_chair_' + image_name)
        cv2.imwrite(save_path, img)

        # Save the new annotations for human on chair
        annotation_save_path = os.path.join(annotations_with_human_on_chair_folder, image_name.replace('.jpg', '.txt'))
        with open(annotation_save_path, 'w') as f:
            for class_id, yolo_bbox in new_annotations:
                f.write(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")

    return len(humans), len(chairs), human_on_chair_count

# Initialize statistics
total_humans = 0
total_chairs = 0
total_humans_on_chairs = 0
total_images_processed = 0

# Process images in each input directory separately
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
for dir_name, input_dir in input_dirs.items():
    dir_total_humans = 0
    dir_total_chairs = 0
    dir_total_humans_on_chairs = 0
    image_paths = [os.path.join(input_dir, img) for img in os.listdir(input_dir) if img.endswith(image_extensions)]
    for image_path in tqdm(image_paths, desc=f"Processing images in {dir_name}"):
        humans, chairs, humans_on_chairs = plot_and_save_image_with_bboxes(image_path)
        dir_total_humans += humans
        dir_total_chairs += chairs
        dir_total_humans_on_chairs += humans_on_chairs
        total_images_processed += 1

    # Print statistics for the current directory
    print(f"\nResults for {dir_name}:")
    print(f"Total images processed: {len(image_paths)}")
    print(f"Total humans detected: {dir_total_humans}")
    print(f"Total chairs detected: {dir_total_chairs}")
    print(f"Total humans on chairs detected: {dir_total_humans_on_chairs}")

    # Update overall statistics
    total_humans += dir_total_humans
    total_chairs += dir_total_chairs
    total_humans_on_chairs += dir_total_humans_on_chairs

# Print final statistics
print("\nOverall Results:")
print(f"Total images processed: {total_images_processed}")
print(f"Total humans detected: {total_humans}")
print(f"Total chairs detected: {total_chairs}")
print(f"Total humans on chairs detected: {total_humans_on_chairs}")

print("Processing completed.")

Processing images in /content/drive/My Drive/GAN/Real: 100%|██████████| 372/372 [10:43<00:00,  1.73s/it]
Processing images in /content/drive/My Drive/Dall_E: 100%|██████████| 131/131 [05:02<00:00,  2.31s/it]
Processing images in /content/drive/My Drive/stable_difuser_images: 100%|██████████| 1064/1064 [39:35<00:00,  2.23s/it]

Total images processed: 1567
Total humans detected: 1950
Total chairs detected: 9614
Total humans on chairs detected: 205
Processing completed.


In [ ]:
total_images = len(raw_images)
total_train_images = len(train_images)
total_val_images = len(val_images)
print(f"Total images: {total_images}")
print(f"Total training images: {total_train_images}")
print(f"Total validation images: {total_val_images}")

In [3]:
import os
import shutil
from tqdm import tqdm
import torch
import cv2

# Load YOLOv5 large model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

# Define paths for each input directory
input_dirs = {
    'GAN/Real': '/content/drive/My Drive/GAN/Real',
    'Dall_E': '/content/drive/My Drive/Dall_E',
    'Stable_Diffuser': '/content/drive/My Drive/stable_difuser_images'
}
new_model_folder = '/content/drive/My Drive/processed_images/verification_model'
images_folder = os.path.join(new_model_folder, 'images')
annotations_folder = os.path.join(new_model_folder, 'annotations')

# Create directories if they don't exist
os.makedirs(images_folder, exist_ok=True)
os.makedirs(annotations_folder, exist_ok=True)

# Define relevant classes for objects that a human can sit on next to a pool
relevant_classes = [0, 56, 57, 58]  # Include humans and objects they can sit on

# Function to convert YOLO bbox to normal bbox
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x = int((x_center - width / 2) * img_width)
    y = int((y_center - height / 2) * img_height)
    w = int(width * img_width)
    h = int(height * img_height)
    return [x, y, w, h]

# Function to convert normal bbox to YOLO bbox
def bbox_to_yolo(bbox, img_width, img_height):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    width = w / img_width
    height = h / img_height
    return [x_center, y_center, width, height]

# Function to check if a human is on a chair
def is_human_on_chair(human_bbox, chair_bbox):
    hx, hy, hw, hh = human_bbox
    cx, cy, cw, ch = chair_bbox
    # Check if the center of the human is within the chair bounding box
    return (cx <= hx + hw / 2 <= cx + cw) and (cy <= hy + hh / 2 <= cy + ch)

# Function to detect objects using YOLOv5 model
def detect_objects(image_path):
    results = model(image_path)
    return results.xyxy[0].cpu().numpy()

# Function to plot bounding boxes on an example image and save it
def plot_and_save_image_with_bboxes(image_path):
    image_name = os.path.basename(image_path)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return 0, 0, 0

    img_height, img_width = img.shape[:2]

    detections = detect_objects(image_path)
    humans = []
    chairs = []

    for det in detections:
        class_id, x1, y1, x2, y2, confidence = int(det[5]), int(det[0]), int(det[1]), int(det[2]), int(det[3]), float(det[4])
        bbox = [x1, y1, x2 - x1, y2 - y1]
        if class_id == 0:
            humans.append(bbox)
        elif class_id in relevant_classes:
            chairs.append(bbox)

    human_on_chair_count = 0
    new_annotations = []

    for human_bbox in humans:
        for chair_bbox in chairs:
            if is_human_on_chair(human_bbox, chair_bbox):
                human_on_chair_count += 1
                cv2.rectangle(img, (human_bbox[0], human_bbox[1]), (human_bbox[0] + human_bbox[2], human_bbox[1] + human_bbox[3]), (255, 0, 0), 3)
                cv2.putText(img, 'Human on Chair', (human_bbox[0], human_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                # Save the new annotation for human on chair
                new_annotations.append((2, bbox_to_yolo(human_bbox, img_width, img_height)))
                break

    if human_on_chair_count > 0:
        # Save the image with bounding boxes and annotations
        save_path = os.path.join(images_folder, image_name)
        cv2.imwrite(save_path, img)

        # Save the new annotations for human on chair
        annotation_save_path = os.path.join(annotations_folder, image_name.replace('.jpg', '.txt'))
        with open(annotation_save_path, 'w') as f:
            for class_id, yolo_bbox in new_annotations:
                f.write(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")

    return len(humans), len(chairs), human_on_chair_count

# Initialize statistics
total_humans = 0
total_chairs = 0
total_humans_on_chairs = 0
total_images_processed = 0

# Process images in each input directory separately
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
for dir_name, input_dir in input_dirs.items():
    dir_total_humans = 0
    dir_total_chairs = 0
    dir_total_humans_on_chairs = 0
    image_paths = [os.path.join(input_dir, img) for img in os.listdir(input_dir) if img.endswith(image_extensions)]
    for image_path in tqdm(image_paths, desc=f"Processing images in {dir_name}"):
        humans, chairs, humans_on_chairs = plot_and_save_image_with_bboxes(image_path)
        dir_total_humans += humans
        dir_total_chairs += chairs
        dir_total_humans_on_chairs += humans_on_chairs
        total_images_processed += 1

    # Print statistics for the current directory
    print(f"\nResults for {dir_name}:")
    print(f"Total images processed: {len(image_paths)}")
    print(f"Total humans detected: {dir_total_humans}")
    print(f"Total chairs detected: {dir_total_chairs}")
    print(f"Total humans on chairs detected: {dir_total_humans_on_chairs}")

    # Update overall statistics
    total_humans += dir_total_humans
    total_chairs += dir_total_chairs
    total_humans_on_chairs += dir_total_humans_on_chairs

# Print final statistics
print("\nOverall Results:")
print(f"Total images processed: {total_images_processed}")
print(f"Total humans detected: {total_humans}")
print(f"Total chairs detected: {total_chairs}")
print(f"Total humans on chairs detected: {total_humans_on_chairs}")

print("Processing completed.")


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirements ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 195.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 200.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0

requirements: AutoUpdate success ✅ 10.2s, installed 3 packages: ['gitpython>=3.1.30', 'pillow>=10.3.0', 'requests>=2.32.0']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



YOLOv5 🚀 2024-5-26 Python-3.10.12 torch-2.3.0+cu121 CPU

100%|██████████| 89.3M/89.3M [00:00<00:00, 348MB/s]

Fusing layers... 
YOLOv5l summary: 367 layers, 46533693 parameters, 0 gradients, 109.0 GFLOPs
Adding AutoShape... 
Processing images in GAN/Real: 100%|██████████| 372/372 [11:23<00:00,  1.84s/it]



Results for GAN/Real:
Total images processed: 372
Total humans detected: 945
Total chairs detected: 2121
Total humans on chairs detected: 75


Processing images in Dall_E: 100%|██████████| 131/131 [04:55<00:00,  2.26s/it]



Results for Dall_E:
Total images processed: 131
Total humans detected: 144
Total chairs detected: 197
Total humans on chairs detected: 41


Processing images in Stable_Diffuser: 100%|██████████| 1064/1064 [39:13<00:00,  2.21s/it]


Results for Stable_Diffuser:
Total images processed: 1064
Total humans detected: 861
Total chairs detected: 7296
Total humans on chairs detected: 89

Overall Results:
Total images processed: 1567
Total humans detected: 1950
Total chairs detected: 9614
Total humans on chairs detected: 205
Processing completed.


In [ ]:
import os
import shutil
from tqdm import tqdm
import torch
import cv2

# Load YOLOv5 large model
model = torch.hub.load('ultralytics/yolov5', 'yolov5l')

# Define paths for each input directory
input_dirs = {
    'GAN/Real': '/content/drive/My Drive/GAN/Real',
    'Dall_E': '/content/drive/My Drive/Dall_E',
    'Stable_Diffuser': '/content/drive/My Drive/stable_difuser_images'
}
new_model_folder = '/content/drive/My Drive/processed_images/verification_model'
images_folder = os.path.join(new_model_folder, 'images')
annotations_folder = os.path.join(new_model_folder, 'annotations')

# Create directories if they don't exist
os.makedirs(images_folder, exist_ok=True)
os.makedirs(annotations_folder, exist_ok=True)

# Define relevant classes for objects that a human can sit on next to a pool
relevant_classes = [0, 56, 57, 58]  # Include humans and objects they can sit on

# Function to convert YOLO bbox to normal bbox
def yolo_to_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x = int((x_center - width / 2) * img_width)
    y = int((y_center - height / 2) * img_height)
    w = int(width * img_width)
    h = int(height * img_height)
    return [x, y, w, h]

# Function to convert normal bbox to YOLO bbox
def bbox_to_yolo(bbox, img_width, img_height):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    width = w / img_width
    height = h / img_height
    return [x_center, y_center, width, height]

# Function to check if a human is on a chair
def is_human_on_chair(human_bbox, chair_bbox):
    hx, hy, hw, hh = human_bbox
    cx, cy, cw, ch = chair_bbox
    # Check if the center of the human is within the chair bounding box
    return (cx <= hx + hw / 2 <= cx + cw) and (cy <= hy + hh / 2 <= cy + ch)

# Function to detect objects using YOLOv5 model
def detect_objects(image_path):
    results = model(image_path)
    return results.xyxy[0].cpu().numpy()

# Function to plot bounding boxes on an example image and save it
def plot_and_save_image_with_bboxes(image_path):
    image_name = os.path.basename(image_path)
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to read image: {image_path}")
        return 0, 0, 0

    img_height, img_width = img.shape[:2]

    detections = detect_objects(image_path)
    humans = []
    chairs = []

    for det in detections:
        class_id, x1, y1, x2, y2, confidence = int(det[5]), int(det[0]), int(det[1]), int(det[2]), int(det[3]), float(det[4])
        bbox = [x1, y1, x2 - x1, y2 - y1]
        if class_id == 0:
            humans.append(bbox)
        elif class_id in relevant_classes:
            chairs.append(bbox)

    human_on_chair_count = 0
    new_annotations = []

    for human_bbox in humans:
        for chair_bbox in chairs:
            if is_human_on_chair(human_bbox, chair_bbox):
                human_on_chair_count += 1
                cv2.rectangle(img, (human_bbox[0], human_bbox[1]), (human_bbox[0] + human_bbox[2], human_bbox[1] + human_bbox[3]), (255, 0, 0), 3)
                cv2.putText(img, 'Human on Chair', (human_bbox[0], human_bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                # Save the new annotation for human on chair
                new_annotations.append((2, bbox_to_yolo(human_bbox, img_width, img_height)))
                break

    if human_on_chair_count > 0:
        # Save the image with bounding boxes and annotations
        save_path = os.path.join(images_folder, image_name)
        cv2.imwrite(save_path, img)

        # Save the new annotations for human on chair
        annotation_save_path = os.path.join(annotations_folder, image_name.replace('.jpg', '.txt').replace('.jpeg', '.txt').replace('.png', '.txt'))
        with open(annotation_save_path, 'w') as f:
            for class_id, yolo_bbox in new_annotations:
                f.write(f"{class_id} {' '.join(map(str, yolo_bbox))}\n")

    return len(humans), len(chairs), human_on_chair_count

# Initialize statistics
total_humans = 0
total_chairs = 0
total_humans_on_chairs = 0
total_images_processed = 0

# Process images in each input directory separately
image_extensions = ('.jpg', '.jpeg', '.png', '.bmp')
for dir_name, input_dir in input_dirs.items():
    dir_total_humans = 0
    dir_total_chairs = 0
    dir_total_humans_on_chairs = 0
    image_paths = [os.path.join(input_dir, img) for img in os.listdir(input_dir) if img.endswith(image_extensions)]
    for image_path in tqdm(image_paths, desc=f"Processing images in {dir_name}"):
        humans, chairs, humans_on_chairs = plot_and_save_image_with_bboxes(image_path)
        dir_total_humans += humans
        dir_total_chairs += chairs
        dir_total_humans_on_chairs += humans_on_chairs
        total_images_processed += 1

    # Print statistics for the current directory
    print(f"\nResults for {dir_name}:")
    print(f"Total images processed: {len(image_paths)}")
    print(f"Total humans detected: {dir_total_humans}")
    print(f"Total chairs detected: {dir_total_chairs}")
    print(f"Total humans on chairs detected: {dir_total_humans_on_chairs}")

    # Update overall statistics
    total_humans += dir_total_humans
    total_chairs += dir_total_chairs
    total_humans_on_chairs += dir_total_humans_on_chairs

# Print final statistics
print("\nOverall Results:")
print(f"Total images processed: {total_images_processed}")
print(f"Total humans detected: {total_humans}")
print(f"Total chairs detected: {total_chairs}")
print(f"Total humans on chairs detected: {total_humans_on_chairs}")

print("Processing completed.")


In [ ]:
# Verify that each image has a corresponding annotation file
missing_annotations = []

for img in os.listdir(images_folder):
    if img.endswith(('.jpg', '.jpeg', '.png')):
        base_name = os.path.splitext(img)[0]
        annotation_file = base_name + '.txt'
        if not os.path.exists(os.path.join(annotations_folder, annotation_file)):
            missing_annotations.append(img)

if missing_annotations:
    print("Images without corresponding annotations:")
    for img in missing_annotations:
        print(img)
else:
    print("All images have corresponding annotations.")
